### Time Adjustment
------------------------------------------------------------

Twitter API returns the date and time that a tweet is published in **UTC**.

To adjust this timezone base on each tweet's location, the following code snippets utilizes the bounding box of coordinates, which enables spatial mapping to tweets' respective city locations, using the **timezonefinder** package. Given that daytime, month, and weekday can be influential factors in twitting about a specific event, and to better utilize the datetime data *(%a %b %d %H:%M:%S %Y)*, we extract *a*: weekday,*b*: month, and *H:* hour fields and stored them as separate features.

In [3]:
#import packages

import json
from timezonefinder import TimezoneFinder
import pytz
import datetime as dt
import pandas as pd
import csv



#need to read the csv file in this way as it had lots of line breaks in some of the columns
with open("File Containing the time data (created_at from twitter).csv") as f:
    reader = csv.reader(f)
    headers = next(reader, None)
    csv_rows = list(reader)
headers


df = pd.DataFrame(csv_rows, columns=headers)

#Creating new columns for the adjusted time
df['local_time_twitterStyle'] = "dummy"
df['local_time_nice'] = "dummy"
df['hour'] = "dummy"
df ['weekday'] = "dummy"
df ['month']= "dummy"
for index, row in df.iterrows():
    try:
        longitude = float(row['place.bounding_box.coordinates.0.0.0'])
        
        latitude = float(row['place.bounding_box.coordinates.0.0.1'])
        
        tf = TimezoneFinder()
        
        #To define the location mapped to the timezone. This location will be used for time adjustment.
        loc = tf.timezone_at(lng=longitude, lat=latitude)  
        
        # Convert string to pytz format
        timezone = pytz.timezone(loc)  
        
        # Convert the UTC time to the local time
        utc_time = dt.datetime.strptime(row['created_at'], '%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC)  # Convert tweet timestamp to datetime object
        # Get local time as datetime object
        local_time = utc_time.replace(tzinfo=pytz.utc).astimezone(timezone)  

        row['local_time_twitterStyle'] = local_time.strftime(format='%a %b %d %H:%M:%S +0000 %Y')
        #print('---------------')
        row['local_time_nice'] = local_time.strftime('%Y-%m-%d %H:%M:%S')
        row['hour']= local_time.strftime('%H')
        row ['weekday']= local_time.strftime('%a')
        row ['month']= local_time.strftime('%m')
        #print(local_time.strftime('%H'))
    except:
            pass



------------------------------